In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import TimeSeriesSplit
from tqdm.notebook import tqdm_notebook
from sklearn import metrics

In [ ]:
provinces = ['BKK','Chiangmai','Khonkaen','Rayong','Saraburi','Surat']
province = provinces[3]
data = pd.read_csv('./'+province+'/train/'+province.lower()+'_train_format_2.csv')

y_train = data.pop('PM2.5')
X_train = data

In [ ]:
tscv = TimeSeriesSplit(n_splits=3)
i = 1
score = []
for tr_index, val_index in tqdm_notebook(tscv.split(X_train)):
    X_tr, X_val = X_train.iloc[tr_index], X_train.iloc[val_index]
    y_tr, y_val = y_train.iloc[tr_index], y_train.iloc[val_index]
    for mf in tqdm_notebook(np.linspace(1, 7, 7)):
        for ne in np.linspace(50, 100, 6):
            for md in np.linspace(20, 40, 5):
                for msl in np.linspace(30, 100, 8):
                    rfr = RandomForestRegressor(
                        max_features=int(mf),
                        n_estimators=int(ne),
                        max_depth=int(md),
                        min_samples_leaf=int(msl),
                        n_jobs = -1)
                    rfr.fit(X_tr, y_tr)
                    y_pred = rfr.predict(X_val)
                    score.append([i,
                                  mf, 
                                  ne,
                                  md, 
                                  msl, 
                                  rfr.score(X_val, y_val),
                                  np.sqrt(metrics.mean_squared_error(y_val, y_pred))])
    i += 1

In [ ]:
import pickle

with open('rf_'+province.lower()+'_score.pickle', 'wb') as fp:
    pickle.dump(score, fp)

In [ ]:
score2 = np.array(score)
print('best parem:',score[np.argmin(score2, axis=0)[-1]]) #please note all these params in note